In [1]:
import pandas as pd
import scipy.io as sio
import numpy as np

from iFEMG_feature import db2mat

In [2]:
db2mat(r"E:\Data\积水潭患者数据集\w8s1\affected", ["bi-0", "bi-05", "bi-1", "bi-2"])

In [8]:
db_file_path = r"E:\Data\积水潭患者数据集\w8s1\affected\bi-2.db"
raw_data = pd.read_table(db_file_path, sep = ';', header = None)
raw_data.iloc[:, 1:17].values.astype(np.float64)


array([[    0.,     0.,  9637., ...,   258.,   281.,   258.],
       [    0.,     0.,  9637., ...,   262.,   276.,   257.],
       [10407.,     0.,  9637., ...,   254.,   271.,   241.],
       ...,
       [ 8920.,     0., 10725., ...,   260.,   273.,   265.],
       [ 8920.,     0., 10725., ...,   262.,   279.,   246.],
       [ 8920.,     0., 10725., ...,   259.,   281.,   239.]])

In [4]:
data = raw_data.iloc[:, 1:17].values
sio.savemat(r"E:\Data\积水潭患者数据集\w8s1\affected\bi_2.mat", {"bi_2": data})

In [5]:
raw_data.shape[1]

18